In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import FormatStrFormatter
import pandas as pd
import re
import string

import time
import os
import sqlite3
from operator import itemgetter
import csv
from bs4 import BeautifulSoup
import nltk
from collections import Counter


In [2]:
base_dir = '../'
data_dir = base_dir + 'data/'

num_of_observation = 350
minimum_claimed = 5
minimum_diff = 10

In [3]:
# Returns a list L, with every element E = L[i] as
# E[0] = product_info , see E[0].keys() for all different info available
# E[1] = product_time_series , see E[1].columns for all different info available

def extract_data_group_by_asin(product_db, display = False):
    conn = sqlite3.connect(product_db)

    status = pd.read_sql_query("SELECT * FROM status", conn)
    
    metadata = pd.read_sql_query("SELECT * FROM metadata", conn)
    metadata = metadata.drop_duplicates((metadata.columns.difference(['msToStart', 'msToEnd', 'recordtime'])))
    
    print("NA rows in status = %d" %np.count_nonzero(status.isnull()))
    print("NA rows in metadata = %d" %np.count_nonzero(metadata.isnull()))
    
    status.dropna(inplace=True)
    metadata.dropna(inplace=True)
    
    conn.close()
    
    if display == True:
        print(status.columns)
#         print(status.head())
        print(metadata.columns)
#         print(metadata.head())
    
    status = status.sort_values(by=['recordtime'])
    
    status_more_than_cutoff = status.groupby("asin").filter(lambda x: len(x) > num_of_observation)
    
    product_list = []
    
    for name, group in status_more_than_cutoff.groupby("asin"):
        group = group.reset_index(drop=True)
        
        if group.iloc[0]['percentClaimed'] < 5 :
            
            if len(np.unique(group['deal_id'].values)) > 1:
                flag = 0
                temp = np.unique(group['deal_id'].values)
                if len(temp) > 2 :
                    print(name)
                for i in temp:
                    idx = group['deal_id'][group['deal_id'] == i].index[0]
                    if (idx > 1):
                        if group.iloc[idx - 1]['msToend'] > 60000:
                            flag = 1
                
                if flag == 1:
                    print("Multiple deal_ids for %s" % name)
                
                
            deal_id = group.iloc[0]['deal_id']
            product_info = metadata.loc[metadata['deal_id'] == deal_id]
            
            if len(product_info) > 1:
                print("Multiple info for deal_id for %s" % deal_id)
            
            if len(product_info) < 1:
                continue
                
            product_list.append([product_info.iloc[0], group])
            
    return product_list
            


In [4]:

# Returns a list L, with every element E = L[i] as
# E[0] = product_info , see E[0].keys() for all different info available
# E[1] = product_time_series , see E[1].columns for all different info available

def extract_data_group_by_deal_id(product_db, display = False):
    conn = sqlite3.connect(product_db)

    status = pd.read_sql_query("SELECT * FROM status", conn)
    
    metadata = pd.read_sql_query("SELECT * FROM metadata", conn)
    metadata = metadata.drop_duplicates((metadata.columns.difference(['msToStart', 'msToEnd', 'recordtime'])))
    
    print("NA rows in status = %d" %np.count_nonzero(status.isnull()))
    print("NA rows in metadata = %d" %np.count_nonzero(metadata.isnull()))
    
    status.dropna(inplace=True)
    metadata.dropna(inplace=True)
    
    conn.close()
    
    if display == True:
        print(status.columns)
#         print(status.head())
        print(metadata.columns)
#         print(metadata.head())
    
    status = status.sort_values(by=['recordtime'])
    
    status_deal_id_groups = status.groupby("deal_id")
    
#     status_more_than_cutoff = status.groupby("asin").filter(lambda x: len(x) > num_of_observation)
    
    product_list = []
    
    for deal_id, deal_group in status_deal_id_groups:
        
        asin_subgroup = deal_group.groupby("asin")
        
        for asin, group in asin_subgroup:
            
            if len(group) < num_of_observation:
                continue
            
            group = group.reset_index(drop=True)

            if group.iloc[0]['percentClaimed'] < minimum_claimed :

                product_info = metadata.loc[metadata['deal_id'] == deal_id]

                if len(product_info) > 1:
                    print("Multiple info for deal_id for %s" % deal_id)

                if len(product_info) < 1:
                    continue

                product_list.append([product_info.iloc[0], group])
            
    return product_list
            


In [5]:
# Returns a list L, with every element E = L[i] as
# E[0] = product_info , see E[0].keys() for all different info available
# E[1] = product_time_series , see E[1].columns for all different info available
# E[2] = List of list. [[a,b,c,d,e], [....] , [....].... n reviews] , [a,b,c,d,e] is review features for first review

def extract_data_group_by_deal_id_ignore_asin(product_db, rev_df, display = False):
    conn = sqlite3.connect(product_db)

    status = pd.read_sql_query("SELECT * FROM status", conn)
    
    metadata = pd.read_sql_query("SELECT * FROM metadata", conn)
    metadata = metadata.drop_duplicates((metadata.columns.difference(['msToStart', 'msToEnd', 'recordtime'])))
    
    print("NA rows in status = %d" %np.count_nonzero(status.isnull()))
    print("NA rows in metadata = %d" %np.count_nonzero(metadata.isnull()))
    
    status.dropna(inplace=True)
    metadata.dropna(inplace=True)
    
    conn.close()
    
    if display == True:
        print(status.columns)
#         print(status.head())
        print(metadata.columns)
#         print(metadata.head())
    
    status = status.sort_values(by=['recordtime'])
    
    status_deal_id_groups = status.groupby("deal_id")
    
#     status_more_than_cutoff = status.groupby("asin").filter(lambda x: len(x) > num_of_observation)
    
    product_list = []
    
    for deal_id, deal_group in status_deal_id_groups:
        
        asin_subgroup = deal_group.groupby("asin")
        
        for asin, group in asin_subgroup:
            
            if len(group) < num_of_observation:
                continue
            
            group = group.reset_index(drop=True)

            if group.iloc[0]['avgpercentage'] < minimum_claimed :

                product_info = metadata.loc[metadata['deal_id'] == deal_id]

                if len(product_info) > 1:
                    print("Multiple info for deal_id for %s" % deal_id)

                if len(product_info) < 1:
                    continue
                
                deal_rev = (rev_df.loc[rev_df['asin'] == asin]).values.tolist()
#                 print(deal_rev)
                
                product_list.append([product_info.iloc[0], group, deal_rev])
                break
            
    return product_list
            


In [6]:
all_rev_df = pd.read_hdf(data_dir + 'html/rev_with_help_r.h5' , key = 'df')

In [7]:
all_rev_df = all_rev_df.drop(['reviewerHist', 'popularity', 'reviewText', 
                              'ADJ', 'ADP', 'ADV', 'CONJ', 'NUM', 'NOUN', 'VERB','X',
                              'prob0', 'prob1', 'reviewer_id'] ,axis=1)

In [8]:
rating_df = all_rev_df[['rating_2', 'rating_3', 'rating_4', 'rating_5']]
rating_df.columns = ['2', '3', '4', '5']
rating_df['1'] = 0
mask = (rating_df==0).all(axis = 1)
rating_df.loc[mask, '1'] = 1
rating_df = rating_df.idxmax(axis=1)

all_rev_df['rating'] = rating_df
all_rev_df = all_rev_df.drop(['rating_2', 'rating_3', 'rating_4', 'rating_5'] ,axis=1)

c:\users\shubham singla\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\shubham singla\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
print(all_rev_df.head())

   num_tokens  num_char  tok_mean   tok_std  wordvec_ratio        asin  \
0        39.0     275.0  6.076923  2.092534       0.925000  B072K67H33   
1         4.0      29.0  6.500000  1.500000       0.800000  B072K67H33   
2        24.0     164.0  5.875000  1.666146       0.840000  B072K67H33   
3         6.0      40.0  5.833333  1.572330       0.857143  B072K67H33   
4        17.0     123.0  6.294118  1.933138       0.944444  B072K67H33   

  cnt_help      date   hr_pred rating  
0        0  20190118  0.698247      4  
1        5  20190118  0.090351      1  
2        0  20190118  0.711443      4  
3        1  20190118  0.105778      2  
4        0  20190118  0.397303      1  


In [10]:
all_rev_df.columns

Index(['num_tokens', 'num_char', 'tok_mean', 'tok_std', 'wordvec_ratio',
       'asin', 'cnt_help', 'date', 'hr_pred', 'rating'],
      dtype='object')

In [11]:
dir_list = os.listdir(data_dir + 'productDB/')

data = []

for dir_name in dir_list:
    if dir_name.isdigit() == False :
        continue
    database = data_dir + 'productDB/' + dir_name + '/product_info.db' 
    print("Date Extraction : %s" % dir_name)
    
    rev_date_df = all_rev_df.loc[all_rev_df['date'] == dir_name]
    print(rev_date_df.shape)
    
    data.extend(extract_data_group_by_deal_id_ignore_asin(database, rev_date_df, display=False))
    
    print("\n\n")

Date Extraction : 20190101
(7419, 10)
NA rows in status = 0
NA rows in metadata = 16



Date Extraction : 20190102
(8305, 10)
NA rows in status = 0
NA rows in metadata = 14



Date Extraction : 20190103
(8101, 10)
NA rows in status = 0
NA rows in metadata = 6



Date Extraction : 20190104
(8018, 10)
NA rows in status = 0
NA rows in metadata = 8



Date Extraction : 20190105
(8718, 10)
NA rows in status = 0
NA rows in metadata = 4



Date Extraction : 20190106
(8178, 10)
NA rows in status = 0
NA rows in metadata = 6



Date Extraction : 20190107
(7511, 10)
NA rows in status = 0
NA rows in metadata = 8



Date Extraction : 20190108
(0, 10)
NA rows in status = 0
NA rows in metadata = 6



Date Extraction : 20190109
(9513, 10)
NA rows in status = 0
NA rows in metadata = 6



Date Extraction : 20190110
(9206, 10)
NA rows in status = 0
NA rows in metadata = 4



Date Extraction : 20190111
(9126, 10)
NA rows in status = 0
NA rows in metadata = 4



Date Extraction : 20190112
(10079, 10)
NA ro

In [12]:
len(data)

26190

In [13]:
np_is_sorted = lambda a: np.all(a[:-1] >= a[1:])

upper_bound_time = 86400000

# Split data into monotonic, non monotonic
def split_data(data):
    
    monotonic_deals = []
    non_monotonic_time_increased = []
    non_monotonic_new_deal = []

    for product_data in data:
        
        # Invalid Time to End
        if np.min(product_data[1]['msToend'].values) < 0 or np.max(product_data[1]['msToend'].values) > upper_bound_time:
            continue
        
        # Percent Claimed is all 0
        if np.all(product_data[1]['avgpercentage'].values == 0):
            continue
        
        # Time Remaining is decreasing
        mono = np_is_sorted(product_data[1]['msToend'].values)
        
        if mono == True:
            monotonic_deals.append(product_data)
            
        else:
            diffs = np.sign(product_data[1]['msToend'].diff())
            idx = np.where(diffs == 1)
            
            # If the percent_claimed_difference at instant of increase is > minimum_diff
            # or the percent claimed at that instant is 0, 
            # then assume a new deal has started
            diff_prod = np.abs(product_data[1]['avgpercentage'].iloc[idx[0][0] - 1] - \
                          product_data[1]['avgpercentage'].iloc[idx[0][0]])
            
            if (diff_prod > minimum_diff) or (product_data[1]['avgpercentage'].iloc[idx[0][0]] == 0):
                non_monotonic_new_deal.append(product_data)
            else:
                non_monotonic_time_increased.append(product_data)
            
    print(len(monotonic_deals))
    print(len(non_monotonic_time_increased))
    print(len(non_monotonic_new_deal))
    
    return monotonic_deals, non_monotonic_time_increased, non_monotonic_new_deal

In [14]:
mono, non_monotonic_time_increased, _ = split_data(data)
print(len(mono))
count = 0
for i in range(len(mono)):
    mono[i][0].to_csv(data_dir + 'WithoutReviews/monotonic_meta/' + str(count) + '.csv', sep=',', encoding='utf-8')
    mono[i][1].to_csv(data_dir + 'WithoutReviews/monotonic/' + str(count) + '.csv', sep=',', encoding='utf-8')
    for j in range (len(mono[i][2])):
        with open(data_dir + 'WithoutReviews/monotonic_meta/' + str(count) + '.csv', "a", newline='') as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(mono[i][2][j])
    if(count%100 == 0):
        print(count)
    count = count + 1

c:\users\shubham singla\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sign


21948
272
72
21948
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
1

In [49]:
np.set_printoptions(threshold=np.nan)
print(non_monotonic_time_increased[0][1][['avgpercentage', 'deal_id', 'msToend']].values)

[[0 'b133498b' 43186041]
 [0 'b133498b' 43121065]
 [0 'b133498b' 43056324]
 [0 'b133498b' 42991780]
 [0 'b133498b' 42926156]
 [0 'b133498b' 42861003]
 [0 'b133498b' 42794066]
 [0 'b133498b' 42728492]
 [0 'b133498b' 42663191]
 [0 'b133498b' 42598563]
 [0 'b133498b' 42532726]
 [0 'b133498b' 42468289]
 [0 'b133498b' 42403941]
 [0 'b133498b' 42337236]
 [0 'b133498b' 42271615]
 [0 'b133498b' 42206839]
 [0 'b133498b' 42142228]
 [0 'b133498b' 42077886]
 [0 'b133498b' 42012805]
 [0 'b133498b' 41947900]
 [0 'b133498b' 41882751]
 [0 'b133498b' 41817985]
 [0 'b133498b' 41752290]
 [0 'b133498b' 41687671]
 [0 'b133498b' 41623104]
 [0 'b133498b' 41557381]
 [0 'b133498b' 41490441]
 [0 'b133498b' 41423079]
 [0 'b133498b' 41357740]
 [0 'b133498b' 41291718]
 [0 'b133498b' 41227303]
 [0 'b133498b' 41160238]
 [0 'b133498b' 41095156]
 [0 'b133498b' 41030836]
 [0 'b133498b' 40963130]
 [0 'b133498b' 40898272]
 [0 'b133498b' 40832559]
 [0 'b133498b' 40767316]
 [0 'b133498b' 40702129]
 [0 'b133498b' 40634169]
